In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from cf_model import MLPCollaborativeFiltering
from dataset import CFDataset, Encoder, split_data
import import_ipynb
import db
from torch.utils.data import DataLoader
from tqdm import tqdm
import random

importing Jupyter notebook from db.ipynb
cf_model.py	    menv			todo.txt
completedstack.txt  model_weights.pth		todo.txt.lock
config.py	    movusers.db			train.ipynb
dataset.py	    one				two
db.ipynb	    __pycache__			user.py
done.txt	    q.py			wandb
done.txt.lock	    qtest.txt			web_spider.py
fdstests.ipynb	    README.md			your_database.db
filemanager.py	    requirements.txt
main.ipynb	    Spider-0_doing_Aquaman.txt


In [2]:
db.display_size()

Users: 2400

Movies: 29

Reviews: 23570


In [3]:
# print(db.display_size())
#TODO change shuffle and not shuffle

data = db.get_table_values("Reviews")
print(data[:10])




list_of_lists = [list(t) for t in data]

# Shuffle the list of lists
random.shuffle(list_of_lists)

# Convert the shuffled list of lists back to a list of tuples
data = [tuple(lst) for lst in list_of_lists]

# print(data[0:10])

SELECT * FROM Reviews
[(1, 1745960, 140182544964272, 'I love it so much, after 36 years I still love the both of them.\n\nMaverick is the one who has believed in his abilities and skills without any limitations Iceman with very short story but I felt the deep friendship between them.\n5 out of 7 found this helpful. Was this review helpful? Sign in to vote.\nPermalink', '10/10', 'June 2022'), (2, 1825683, 140182544964272, "Terribly bad and boring. I'm guessing the only reason this is rated so high is because of the blind fanboys and girls.\n6 out of 13 found this helpful. Was this review helpful? Sign in to vote.\nPermalink", '1/10', 'May 2018'), (3, 10872600, 140182544964272, 'Tom Holland delivers once again in this crazy marvel movie! I love the way he acts, and this movie might be better than the first 3 movies of spiderman(by raimy) i think most people will be entertained by the fun and storyline, it is a great continuation of the spiderman universe and one of the best films in many

In [4]:
encoder = Encoder([did[2] for did in data],[did[1] for did in data])
# print(encoder.user_to_idx)
# print(encoder.idx_to_user)

In [5]:
all_ds = CFDataset(data,encoder)

In [6]:
train_data, test_data = split_data(data)
train_data, validation_data = split_data(train_data)
train_ds = CFDataset(train_data,encoder)
test_ds = CFDataset(test_data,encoder)
validation_ds = CFDataset(validation_data,encoder)


BATCH_SIZE = 64
train_data_loader = DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
validation_data_loader = DataLoader(validation_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
test_data_loader = DataLoader(test_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)

In [7]:
all_ds[20]

(tensor(20), tensor(4), tensor(2.))

In [8]:
# for ds in train_data_loader:
#     print(ds)

In [9]:
max([ds[1] for ds in train_ds.dataset])

tensor(9)

In [10]:
num_users = len(db.get_table_values("Users"))
num_movies = len(db.get_table_values("Movies"))
FEATURES=700
model = MLPCollaborativeFiltering(num_users, num_movies, embedding_dim=FEATURES)


LEARNING_RATE = 0.001
criterion = nn.MSELoss()
#weight decay L2 regularization
# optimiser = optim.SGD(model.parameters(), lr=0.001,weight_decay=1e-5)\
optimiser = optim.SGD(model.parameters(), lr=LEARNING_RATE,weight_decay=0.1)
EPOCHS = 1000

SELECT * FROM Users
SELECT * FROM Movies


# PRETEST

In [11]:
#test data
model.eval()  # Switch to evaluation mode
test_loss = 0
correct_predictions = 0  # Fixed variable name for clarity
total_contexts = 0

with torch.no_grad():  # Disable gradient computation
    for user_ids, movie_ids, ratings in test_data_loader:
        predictions = model(user_ids,movie_ids)  # Generate predictions
        # Calculate and accumulate loss
        loss = criterion(predictions, ratings)
        test_loss += loss.item()
        
        # # Reshape predictions to match [batch_size, context_size, vocab_size]
        # predictions = predictions.view(-1, context_size, VOCAB_SIZE)
        
        # # Get top prediction for each context position
        # top_predictions = predictions.argmax(dim=2)
        
        # # Calculate correct predictions
        # correct_preds = (top_predictions == context).float().sum()
        # correct_predictions += correct_preds.item()  # Accumulate correct predictions
        
        # total_contexts += context.numel()  # Total number of context word positions evaluated

# Calculate final metrics
test_loss /= len(test_data_loader)  # Average loss per batch
# print('correct predictions = ',correct_predictions)
# print('out of  = ',total_contexts)
# accuracy = correct_predictions / total_contexts  # Compute accuracy

# print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')

# Evaluate the model
# (This would involve using a separate validation set or performing cross-validation)
test_loss

59.473445317516585

## wandb

In [12]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    entity="jcrich",
    project="collaborative filter model",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LEARNING_RATE,
    "architecture": "collaborative filter",
    "dataset": "imdb",
    "epochs": EPOCHS,
    }
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jcrich. Use `wandb login --relogin` to force relogin


In [ ]:
# Initialize lists to keep track of losses and epochs
train_losses = []
validation_losses = []
epochs = []

#get it at it's finest....
best_val_loss = float('inf')

for epoch in range(EPOCHS):
    model.train()
    total_epoch_loss = 0
    for batch_idx, (user_ids, movie_ids, ratings) in tqdm(enumerate(train_data_loader), total=len(train_data_loader), desc=f'Epoch {epoch}'):

        optimiser.zero_grad()
        predictions = model(user_ids, movie_ids)
        loss = criterion(predictions, ratings)

        loss.backward()
        optimiser.step()
        
        batch_loss = loss.item()
        total_epoch_loss += batch_loss
        
    # After all batches, calculate average loss for the epoch
    avg_epoch_loss = total_epoch_loss / len(train_data_loader)
    train_losses.append(avg_epoch_loss)
    epochs.append(epoch)  # Append the current epoch number
    
    wandb.log({"Training Loss": avg_epoch_loss, "Epoch": epoch})
    # print(f'Epoch: {epoch}, Average Training Loss: {avg_epoch_loss}')

    # Start validation phase
    model.eval()  # Set the model to evaluation mode
    total_validation_loss = 0
    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        for batch_idx, (user_ids, movie_ids, ratings) in enumerate(validation_data_loader):
            predictions = model(user_ids, movie_ids)
            loss = criterion(predictions, ratings)
            total_validation_loss += loss.item()

    
    
    # Calculate and log validation loss after the epoch
    avg_validation_loss = total_validation_loss / len(validation_data_loader)
    validation_losses.append(avg_validation_loss)

    # Save the model if this epoch has the best validation loss so far
    if avg_validation_loss < best_val_loss:
        best_val_loss = avg_validation_loss
        # Save model state
        torch.save(model.state_dict(), 'best_model_state.pth')
        # If you also want to save the optimizer state along with the model:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_validation_loss,
        }, 'best_model_checkpoint.pth')
    
    wandb.log({"Validation Loss": avg_validation_loss, "Epoch": epoch})
    # print(f'Epoch: {epoch}, Average Validation Loss: {avg_validation_loss}')    
    # After training and validation phases, log the losses with the epoch number
    wandb.log({"Epoch": epoch, "Training Loss": avg_epoch_loss, "Validation Loss": avg_validation_loss})



wandb.finish()

Epoch 818:  66%|███████████████████▏         | 155/235 [00:00<00:00, 185.79it/s]

In [ ]:
#test data
model.eval()  # Switch to evaluation mode
test_loss = 0
correct_predictions = 0  # Fixed variable name for clarity
total_contexts = 0

with torch.no_grad():  # Disable gradient computation
    for user_ids, movie_ids, ratings in test_data_loader:
        predictions = model(user_ids,movie_ids)  # Generate predictions
        # Calculate and accumulate loss
        loss = criterion(predictions, ratings)
        test_loss += loss.item()
        
        # # Reshape predictions to match [batch_size, context_size, vocab_size]
        # predictions = predictions.view(-1, context_size, VOCAB_SIZE)
        
        # # Get top prediction for each context position
        # top_predictions = predictions.argmax(dim=2)
        
        # # Calculate correct predictions
        # correct_preds = (top_predictions == context).float().sum()
        # correct_predictions += correct_preds.item()  # Accumulate correct predictions
        
        # total_contexts += context.numel()  # Total number of context word positions evaluated

# Calculate final metrics
test_loss /= len(test_data_loader)  # Average loss per batch
# print('correct predictions = ',correct_predictions)
# print('out of  = ',total_contexts)
# accuracy = correct_predictions / total_contexts  # Compute accuracy

# print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')

# Evaluate the model
# (This would involve using a separate validation set or performing cross-validation)
test_loss

## Preference ranking

In [ ]:
target_user_id = 140440102666832

In [ ]:
e_uid = encoder.encode(target_user_id,encoder.user_to_idx)

In [ ]:
def get_user_features(uid):

    for i in range(0,len(all_ds)):

        u,m,r = all_ds[i]
        # print(u)
        if u == uid:
            yield [m,r]
    

In [ ]:
def predict_ratings(e_uid,e_mid,model):
    e_uid_tensor = torch.tensor(e_uid, dtype=torch.int64).unsqueeze(0)
    movie_eid_tensor = torch.tensor(e_mid,dtype=torch.int64).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        user_rating = model(e_uid_tensor,movie_eid_tensor)
        return movie_eid_tensor,user_rating
        
    

In [ ]:
#get user and features
seen_features = set(tuple(fe) for fe in get_user_features(e_uid)) # Get user features if needed
seen_movies = set(sf[0].item() for sf in seen_features)
all_movies = set([num for num in range(30)])
unseen_movies = all_movies - seen_movies
unseen_features = {tuple(predict_ratings(e_uid,um,model)) for um in unseen_movies}
all_features = seen_features | unseen_features

sorted_set = sorted(all_features, key=lambda x: x[1])
top_n = 10
recommendations = list(filter(lambda x: x[0].item() in unseen_movies,sorted_set))
n_recommendations = list(map(lambda x: x[0],recommendations[-1:top_n*-1:-1]))
n_recommendations


# Matching

In [ ]:
#all users, with all movie ratings.
def generate_all_ratings(model,num_users,num_movies):
    #build matrix

    #data already encoded....
    all_data = [['dummy',mnm,unm,'dummy','0/10'] for unm in range(0,num_users) for mnm in range(0,num_movies)]
    # print(all_data)
    # print([em for em in all_data])
    fake_encoder = Encoder([did[2] for did in all_data],[did[1] for did in all_data])
    all_ds = CFDataset(all_data,fake_encoder)
    
    data_loader = DataLoader(all_ds, batch_size=1)
    for uid,mid,ra in data_loader:
        # print(uid,mid)
        prediction = model(uid,mid)
        yield (uid.item(),mid.item(),prediction.item())



In [ ]:
def create_ratings_tensor(data, num_users, num_movies):
    # Initialize an empty tensor to hold the ratings
    ratings_tensor = torch.zeros(num_users, num_movies)
    
    # Iterate over the data and fill the tensor
    for entry in data:
        user_id, movie_id, rating = entry
        # Convert rating to float
        rating = float(rating)
        ratings_tensor[user_id, movie_id] = rating
    
    return ratings_tensor

In [ ]:
# Step 1: Generate Predictions
predictions = [ra for ra in generate_all_ratings(model,num_users,num_movies)]
# print(predictions)
ratings_tensor = create_ratings_tensor(predictions,num_users,num_movies)
print(ratings_tensor)
# predictions[0]

# ratings_tensor.shape

# ratings_tensor[0]

import torch.nn.functional as F

# Assuming predicted_ratings is your tensor of predicted ratings
# predicted_ratings.shape should be (num_users, num_movies)

# # Calculate cosine similarity
# user_similarities = F.cosine_similarity(ratings_tensor, ratings_tensor, dim=1)

# user_similarities

## COSINE SIMILARITY

In [ ]:
# Step 1: Normalize predicted ratings
normalized_ratings = F.normalize(ratings_tensor, p=2, dim=1)

# Step 2: Calculate cosine similarity
user_similarities = torch.matmul(normalized_ratings, normalized_ratings.T)

# Set diagonal elements to a large negative value to exclude self-similarity
user_similarities.fill_diagonal_(-float('inf'))

# You can optionally convert the similarities tensor to a numpy array for easier manipulation
user_similarities_np = user_similarities.numpy()

# Print or use the user similarities tensor
print(user_similarities)

### Matches

In [ ]:
top_matches = {}
for i in range(len(user_similarities)):
    # Sort similarities for the current user i
    ranked_users = torch.argsort(torch.tensor(user_similarities[i]), descending=True)
    # Exclude self from top matches (optional)
    top_matches[i] = ranked_users[ranked_users != i]


In [ ]:
top_matches

In [ ]:
encoder.decode(71,encoder.idx_to_user)

In [ ]:
#decoded

decoded_matches = {encoder.decode(k,encoder.idx_to_user):[encoder.decode(ve,encoder.idx_to_user) for ve in v.tolist()] for k,v in top_matches.items()}
decoded_matches

In [ ]:
# Step 4: Select Top Matches
N = 5  # Number of top matches to select
for user, similar_users in decoded_matches.items():
    decoded_matches[user] = similar_users[:N]

decoded_matches

In [ ]:
#decode....

## FINALLY FIND THE MATCHES....

In [ ]:
target_user = 140440102666832



In [ ]:
decoded_matches[target_user]

In [ ]:
neighbour = 140440115331424

## Now see how they relate to the database...

In [ ]:
reviews = db.get_table_values('Reviews')

In [ ]:
#TODO

In [ ]:
def reviews_info(user_id,reviews):
    # print(db.get_table_values('Users'))
    info = {}
    info[user_id] = list(filter(lambda x :x[0] == user_id,db.get_table_values('Users')))[0][1]

    
    #TODO finish work...
    mov_tab = db.get_table_values('Movies')
    
    for review in reviews:
        mov_id = review[1]
        info[mov_id] = {'title':list(filter(lambda x :x[0] == mov_id,mov_tab))[0][1],'rating':review[4]}
        
    return info
    
    

In [ ]:
target_reviews = list(filter(lambda rv: rv[2] == target_user,reviews))
neighbour_reviews = list(filter(lambda rv: rv[2] == neighbour,reviews))

In [ ]:
tar_info = reviews_info(target_user,target_reviews)
neigh_info = reviews_info(neighbour,neighbour_reviews)

In [ ]:
tar_info

In [ ]:
neigh_info

# SAVE_MODEL

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')